# Exercise 04

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

# Importing Libararies

In [5]:
import tensorflow as tf
import os
import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Download Data Set

In [2]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2021-09-15 08:29:26--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-09-15 08:29:26 (121 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
zipref = zipfile.ZipFile('/tmp/happy-or-sad.zip', 'r')
zipref.extractall('/tmp/h-or-s')
zipref.close()

In [4]:
!ls /tmp/h-or-s/

happy  sad


# Preprocess Dataset

In [25]:
base_dir = '/tmp/h-or-s/'
print(base_dir)

train_data_gen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_data_gen.flow_from_directory(directory=base_dir,
                                                     target_size=(150, 150),
                                                     class_mode='binary',
                                                     batch_size=10)


/tmp/h-or-s/
Found 80 images belonging to 2 classes.


In [22]:
train_generator

# Create the Model

In [15]:
DESIRED_ACCURACY = 0.999


class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > DESIRED_ACCURACY:
      print('Reached 99.9% accuracy, cancelling training')
      self.model.stop_training = True


callbacks = myCallbacks() 

In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001),
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9248)             

In [29]:
history = model.fit(train_generator,
                    epochs=15,
                    steps_per_epoch=8,
                    callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 3s 220ms/step - loss: 2.1812 - accuracy: 0.4625
Epoch 2/15
8/8 [==============================] - 2s 211ms/step - loss: 0.4171 - accuracy: 0.7875
Epoch 3/15
8/8 [==============================] - 2s 213ms/step - loss: 0.1954 - accuracy: 0.9500
Epoch 4/15
8/8 [==============================] - 2s 209ms/step - loss: 0.1325 - accuracy: 0.9625
Epoch 5/15
8/8 [==============================] - 2s 211ms/step - loss: 0.0914 - accuracy: 0.9500
Epoch 6/15
8/8 [==============================] - 2s 216ms/step - loss: 0.0644 - accuracy: 0.9750
Epoch 7/15
8/8 [==============================] - 2s 214ms/step - loss: 0.0263 - accuracy: 0.9875
Epoch 8/15
8/8 [==============================] - 2s 211ms/step - loss: 0.0359 - accuracy: 0.9875
Epoch 9/15
8/8 [==============================] - 2s 211ms/step - loss: 0.0072 - accuracy: 1.0000
Reached 99.9% accuracy, cancelling training
